# Afriat

In [1]:
import warnings
import os
import sys
import gc
import warnings

In [37]:
import anndata as ad
import scanpy as sc
import copy
import torch
from pathlib import Path
import networkx as nx
from sklearn.neighbors import kneighbors_graph
import numpy as np
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.stats
import tqdm

sys.path.append("/home/icb/kemal.inecik/work/codes/tardis")
import tardis
from tardis._utils.functions import create_random_mask
from tardis._utils.warnings import suppress_stdout
import sklearn
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
tardis.config = tardis.config_server
print(f"CUDA used: {torch.cuda.is_available()}")

CUDA used: True


In [3]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
_rcparams_path = "/home/icb/kemal.inecik/work/codes/tardis/training/local/figures/rcparams.pickle"
with open(_rcparams_path, 'rb') as file:
    _rcparams = pickle.load(file)
plt.rcParams.update(_rcparams)

In [33]:
def read_adata():
    adata_file_path = os.path.join(tardis.config.io_directories["processed"], "biolord_afriat.h5ad")
    assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
    adata = ad.read_h5ad(adata_file_path)
    return adata

In [50]:
# load the pickle

In [60]:
metrics = []

In [61]:
with suppress_stdout(), warnings.catch_warnings():
    warnings.simplefilter("ignore")
    metric = dict()
    model_name = "afriat_v2_4_1"
    metric["model_name"] = model_name
    metric["model_desc"] = "together"
    adata = read_adata()
    dir_path = os.path.join(
        tardis.config.io_directories["models"],
        model_name
    )
    vae = tardis.MyModel.load(dir_path=dir_path, adata=adata)
    from tardis._disentanglementmanager import DisentanglementManager as DM
    adata.obs["validation"] = "train"
    adata.obs["validation"].iloc[vae.validation_indices] = "validation"

    metric.update({"r2": vae.get_reconstruction_r2_training(top_n=[])})
    metric.update({"mi_zone": vae.get_MI_normalized_training("zone")})
    metric.update({"mi_status_control": vae.get_MI_normalized_training("status_control")})
    metric.update({"mi_time_cat": vae.get_MI_normalized_training("time_cat")})
    metric.update({"knn_zone": vae.get_knn_purity_training("zone")})
    metric.update({"knn_time_cat": vae.get_knn_purity_training("time_cat")})
    metric.update({"knn_status_control": vae.get_knn_purity_training("status_control")})

    data = vae.get_latent_representation()
    factors_indices = sorted([[indice, config.obs_key] for config in DM.configurations.items for indice in config.reserved_latent_indices])
    factors = np.vstack([sklearn.preprocessing.LabelEncoder().fit_transform(
        adata.obs[j].copy().astype(str).values.flatten()
    ) for i, j in factors_indices]).T
    data = data[:, [i for i, j in factors_indices]]
    
    metric.update({"maxmig": {
        "all": np.mean(vae.get_MI_precalculated(data=data, factors=factors, variation="maxMIG", discretization_bins=256)),
        "validation": np.mean(vae.get_MI_precalculated(data=data[vae.validation_indices, :], factors=factors[vae.validation_indices, :], variation="maxMIG", discretization_bins=256)),
        "train": np.mean(vae.get_MI_precalculated(data=data[vae.train_indices, :], factors=factors[vae.train_indices, :], variation="maxMIG", discretization_bins=256))
    }})

    metrics.append(metric)
    del adata, vae, DM, model_name, metric, dir_path, factors_indices, factors, data
    gc.collect()

In [62]:
with suppress_stdout(), warnings.catch_warnings():
    warnings.simplefilter("ignore")
    adata = read_adata()
    metric = dict()
    metric["model_name"] = "afriat_v1_1;afriat_v3_1;afriat_v4_1"
    metric["model_desc"] = "status_control;zone;time_cat"
    vaes = dict()
    comparisons_path = os.path.join(
        tardis.config.io_directories["processed"], "figures",
        "maxmig_models.pickle"
    )
    # Load dictionary from a pickle file
    with open(comparisons_path, 'rb') as f:
        metrics.extend(pickle.load(f))
    dir_path = os.path.join(
        tardis.config.io_directories["models"],
        "afriat_v1_1"
    )
    vaes["status_control"] = tardis.MyModel.load(dir_path=dir_path, adata=adata)
    dir_path = os.path.join(
        tardis.config.io_directories["models"],
        "afriat_v3_1"
    )
    vaes["zone"] = tardis.MyModel.load(dir_path=dir_path, adata=adata)
    dir_path = os.path.join(
        tardis.config.io_directories["models"],
        "afriat_v4_1"
    )
    vaes["time_cat"] = tardis.MyModel.load(dir_path=dir_path, adata=adata)

    for k in vaes:
        adata.obs[f"validation_{k}"] = "train"
        adata.obs[f"validation_{k}"].iloc[vaes[k].validation_indices] = "validation"

    factors_indices = sorted([[k, i] for k in vaes for i in range(8)])
    datas = {j: vaes[j].get_latent_representation() for j in vaes}
    factors_indices_obs = sorted({i for i, j in factors_indices})

    factors = np.vstack([sklearn.preprocessing.LabelEncoder().fit_transform(
        adata.obs[j].copy().astype(str).values.flatten()
    ) for j, i in factors_indices]).T
    data = np.hstack([datas[d][:, range(8)] for d in factors_indices_obs])
    vae = vaes["zone"]
    metric.update({"maxmig": {
        "all": np.mean(vae.get_MI_precalculated(data=data, factors=factors, variation="maxMIG", discretization_bins=256)),
        "validation_zone": np.mean(vae.get_MI_precalculated(data=data[vaes["zone"].validation_indices, :], factors=factors[vaes["zone"].validation_indices, :], variation="maxMIG", discretization_bins=256)),
        "train_zone": np.mean(vae.get_MI_precalculated(data=data[vaes["zone"].train_indices, :], factors=factors[vaes["zone"].train_indices, :], variation="maxMIG", discretization_bins=256)),
        "validation_status_control": np.mean(vae.get_MI_precalculated(data=data[vaes["status_control"].validation_indices, :], factors=factors[vaes["status_control"].validation_indices, :], variation="maxMIG", discretization_bins=256)),
        "train_status_control": np.mean(vae.get_MI_precalculated(data=data[vaes["status_control"].train_indices, :], factors=factors[vaes["status_control"].train_indices, :], variation="maxMIG", discretization_bins=256)),
        "validation_time_cat": np.mean(vae.get_MI_precalculated(data=data[vaes["time_cat"].validation_indices, :], factors=factors[vaes["time_cat"].validation_indices, :], variation="maxMIG", discretization_bins=256)),
        "train_time_cat": np.mean(vae.get_MI_precalculated(data=data[vaes["time_cat"].train_indices, :], factors=factors[vaes["time_cat"].train_indices, :], variation="maxMIG", discretization_bins=256)),
    }})
    metrics.append(metric)
    del dir_path, adata, vae, vaes, datas, factors_indices, factors_indices_obs, factors, data, metric
    gc.collect()

In [63]:
with suppress_stdout(), warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    for model_name, model_desc in [["afriat_v1_1", "status_control"], ["afriat_v3_1", "zone"], ["afriat_v4_1", "time_cat"]]:
        adata = read_adata()
        metric = dict()
        metric["model_name"] = model_name
        metric["model_desc"] = model_desc
    
        dir_path = os.path.join(
            tardis.config.io_directories["models"],
            model_name
        )
        vae = tardis.MyModel.load(dir_path=dir_path, adata=adata)

        from tardis._disentanglementmanager import DisentanglementManager as DM
        adata.obs["validation"] = "train"
        adata.obs["validation"].iloc[vae.validation_indices] = "validation"
    
        metric.update({"r2": vae.get_reconstruction_r2_training(top_n=[])})
        metric.update({f"mi_{model_desc}": vae.get_MI_normalized_training(model_desc)})
        metric.update({f"knn_{model_desc}": vae.get_knn_purity_training(model_desc)})
        
        metrics.append(metric)
        del adata, vae, DM, model_name, model_desc, metric, dir_path
        gc.collect()

# tidy-up

In [76]:
metrics

[{'model_name': 'afriat_v2_4_1',
  'model_desc': 'together',
  'r2': {('validation', 'total'): 0.9993884732160239,
   ('train', 'total'): 0.9994934378674564},
  'mi_zone': {('validation', 'reserved'): 0.9984358727426297,
   ('validation', 'unreserved'): 0.10113067049437616,
   ('train', 'reserved'): 0.998713989002184,
   ('train', 'unreserved'): 0.05535010718423551},
  'mi_status_control': {('validation', 'reserved'): 0.9917252897308235,
   ('validation', 'unreserved'): 0.20399079256612668,
   ('train', 'reserved'): 0.9928320402813209,
   ('train', 'unreserved'): 0.14720987167280009},
  'mi_time_cat': {('validation', 'reserved'): 0.9692151235353923,
   ('validation', 'unreserved'): 0.1806223281902898,
   ('train', 'reserved'): 0.9677143943771795,
   ('train', 'unreserved'): 0.09944049185109856},
  'knn_zone': {('validation', 'reserved'): 1.0,
   ('validation', 'unreserved'): 0.584681952683234,
   ('train', 'reserved'): 1.0,
   ('train', 'unreserved'): 0.5804389831709783},
  'knn_time_c